In [1]:
import pandas as pd
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from tqdm import tqdm

from utils import get_file_paths
from vignette_creation import convolution

import json
with open('config.json') as f_in:
    config = json.load(f_in)

In [2]:
rgb_imgs, masks = get_file_paths()

In [15]:
df = pd.DataFrame(columns=config['stat_vignettes_col'])

In [16]:
for idx, img in tqdm(enumerate(rgb_imgs)):
    crop_rgb = cv2.imread(img)
    crop_mask = cv2.imread(masks[idx])[:,:,0]

    rgb_name = os.path.basename(img)
    dept = rgb_name[4:6]

    # Calculate characteristics of the RGB crop and add to the DataFrame
    mask_2 = np.stack((crop_mask,)*3, axis=-1)
    non_mask = np.logical_not(mask_2)
    non_crop_mask = np.logical_not(crop_mask)
    
    res = [dept, rgb_name, crop_mask.sum()] + \
    np.mean(crop_rgb, axis=(0, 1), where=mask_2.astype(bool)).tolist() + \
    np.mean(crop_rgb, axis=(0, 1), where=non_mask.astype(bool)).tolist() + \
    np.std(crop_rgb, axis=(0, 1), where=mask_2.astype(bool)).tolist() + \
    np.std(crop_rgb, axis=(0, 1), where=non_mask.astype(bool)).tolist() + \
    [np.mean(convolution(crop_rgb[:, :, 0], N=3), where=crop_mask.astype(bool)), np.mean(convolution(crop_rgb[:, :, 1], N=3), where=crop_mask.astype(bool)), np.mean(convolution(crop_rgb[:, :, 2], N=3), where=crop_mask.astype(bool))] + \
    [np.mean(convolution(crop_rgb[:, :, 0], N=3), where=non_crop_mask.astype(bool)), np.mean(convolution(crop_rgb[:, :, 1], N=3), where=non_crop_mask.astype(bool)), np.mean(convolution(crop_rgb[:, :, 2], N=3), where=non_crop_mask.astype(bool))] + \
    [np.std(convolution(crop_rgb[:, :, 0], N=3), where=crop_mask.astype(bool)), np.std(convolution(crop_rgb[:, :, 1], N=3), where=crop_mask.astype(bool)), np.std(convolution(crop_rgb[:, :, 2], N=3), where=crop_mask.astype(bool))] + \
    [np.std(convolution(crop_rgb[:, :, 0], N=3), where=non_crop_mask.astype(bool)), np.std(convolution(crop_rgb[:, :, 1], N=3), where=non_crop_mask.astype(bool)), np.std(convolution(crop_rgb[:, :, 2], N=3), where=non_crop_mask.astype(bool))] 
    # vignette_data.append(res)

    # Append the list to the DataFrame
    df.loc[len(df) + len(set(range(df.index[-1]))-set(df.index))
    if len(df.index) > 0
    else 0] = res

46584it [1:21:46,  9.49it/s]


In [19]:
df.to_csv('vignettes/stat_vignettes_bis.csv')